In [2]:
import time
import mne
from esinet.minimum_norm import mne_eloreta
from esinet.simulation import Simulation
from esinet.forward import create_forward_model, get_info
from esinet.net import Net
from esinet.util import wrap_mne_inverse, unpack_fwd

info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)
pos = unpack_fwd(fwd)[2]


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.9s remaining:    0.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished


In [3]:
settings = dict(duration_of_trial=0.2, number_of_sources=20)
sim = Simulation(fwd, info, settings=settings, parallel=False)

n_samples = 1
sim.simulate(n_samples=n_samples)

epochs = sim.eeg_data[0]
data = epochs.get_data()
print(data.shape)

Simulating data based on sparse patches.


100%|██████████| 1/1 [00:00<00:00, 71.62it/s]

(1, 61, 20)


In [4]:
import numpy as np
from mne.channels.layout import _find_topomap_coords

def elec_transformer(pos):
    def trans(vec):
        im, _ = mne.viz.plot_topomap(vec, pos, show=False, res=12)
        img = np.asarray(im.get_array()[::-1])
        return img

    return trans
elec_pos = _find_topomap_coords(epochs.info, epochs.ch_names)
foo = elec_transformer(elec_pos)

result = foo(np.random.randn(61,))

In [ ]:
n  = 100
arr = np.random.randn(61)
start = time.time()
for _ in range(n):
    foo(arr)
end = time.time()
print((end-start)/n)

In [ ]:
import matplotlib.pyplot as plt

# epochs.info
start = time.time()
im, _ = mne.viz.plot_topomap(data[0,:, 0], elec_pos, show=False, res=12)
# img = im.get_array()[::-1]
img = np.asarray(im.get_array()[::-1])
end = time.time()
print((end-start), ' s')
plt.figure()
plt.imshow(img)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential
n_filters = 8
kernel_size = (3, 3)
n_units = 512
activation = 'relu'
output_shape = sim.source_data[0].data.shape[0]

model = Sequential()
model.add(Conv2D(n_filters, kernel_size, activation=activation))
model.add(Flatten())
model.add(Dense(n_units, activation=activation))
model.add(Dense(output_shape, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')



In [ ]:
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
n = 100

y_true = np.random.randn(n, 20)
y_pred = y_true + np.random.randn(n, 20)*0.5

corrs = np.zeros(n)
cosines = np.zeros(n)
mses = np.zeros(n)
nmses = np.zeros(n)

for i, (t, p) in enumerate(zip(y_true, y_pred)):
    mses[i] = np.mean((t-p)**2)
    nmses[i] = np.mean((t/np.max(np.abs(t)) - p/np.max(np.abs(p)))**2)
    corrs[i] = pearsonr(t, p)[0]
    cosines[i] = cosine(t,p)
    


# Faster Implementation

## Some little more data

In [5]:
settings = dict(duration_of_trial=0.2, number_of_sources=20)
sim = Simulation(fwd, info, settings=settings, parallel=False)

n_samples = 1
sim.simulate(n_samples=n_samples)

epochs = sim.eeg_data[0]
data = epochs.get_data()[0,:, 0]


Simulating data based on sparse patches.


100%|██████████| 1/1 [00:00<00:00, 334.26it/s]


In [6]:
from mne.viz.topomap import _setup_interp, _make_head_outlines, _check_sphere, _check_extrapolate

def make_interpolator(elec_pos, res=9, ch_type='eeg'):
    extrapolate = _check_extrapolate('auto', ch_type)
    sphere = sphere = _check_sphere(None)
    outlines = 'head'
    outlines = _make_head_outlines(sphere, elec_pos, outlines, (0., 0.))
    border = 'mean'
    extent, Xi, Yi, interpolator = _setup_interp(
        elec_pos, res, extrapolate, sphere, outlines, border)
    interpolator.set_locations(Xi, Yi)

    return interpolator

elec_pos = _find_topomap_coords(epochs.info, epochs.ch_names)
interpolator = make_interpolator(elec_pos)
topo = interpolator.set_values(data)()[::-1]



In [8]:
np.nanmean(topo**2)

2.4050555355051252e-12

In [16]:
mat = np.random.randn(100, 20, 10)
mat[0,0,0] = np.nan
mat[0,0,1] = np.nan
# mat[np.isnan(mat)] = [0]

In [20]:
mat[np.isnan(mat)] = 0
mat[np.isnan(mat)]

array([], dtype=float64)

In [29]:
from copy import deepcopy
from tqdm.notebook import tqdm
n = 100
data = [np.random.randn(100, 61) for _ in range(n)]
dat_interp = deepcopy(data)
for i, sample in tqdm(enumerate(data)):
    list_of_time_slices = []
    for time_slice in sample:
        time_slice_interp = interpolator.set_values(time_slice)()[::-1]
        list_of_time_slices.append(time_slice_interp)
    dat_interp[i] = np.stack(list_of_time_slices, axis=0)


0it [00:00, ?it/s]

In [32]:
dat_interp[0].shape

(100, 9, 9)

In [1]:
%matplotlib qt
plt.figure()
plt.subplot(311)
plt.imshow(Xi)
plt.subplot(312)
plt.imshow(Yi)
plt.subplot(313)
plt.imshow(topo)

plt.figure()
mne.viz.plot_topomap(data, elec_pos, show=True, res=12)


NameError: name 'plt' is not defined

## Speed

In [37]:
from tqdm.notebook import tqdm
n = 1000
elec_pos = _find_topomap_coords(epochs.info, epochs.ch_names)
interpolator = make_interpolator(elec_pos)
start = time.time()
for _ in tqdm(range(n)):
    topo = interpolator.set_values(data)()[::-1]
end = time.time()
print((end-start)/n)

n_total = 11000*110
time_s = n_total * (end-start)/n
time_m = (n_total * (end-start)/n) / 60
print(time_s, time_m)


  0%|          | 0/1000 [00:00<?, ?it/s]

0.0006153542995452881
744.5787024497986 12.409645040829977


In [ ]:
import multiprocessing as mp
n_total = 11000*110
datas = [data for _ in range(n_total)]

pool = mp.Pool(mp.cpu_count())
data_interp = [pool.apply(interpolator.set_values, args=(d,)) for d in datas]


In [18]:
from parmapper import parmapper as pmap
n_total = 11000*110
interpolator = make_interpolator(elec_pos)

datas = [data for _ in range(n_total)]
result = pmap(interpolator.set_values, datas)

In [19]:
list(result)

RuntimeError: Not compatible with Windows